In [69]:
import numpy as np
import pandas as pd

Import new dataset

In [70]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
anime = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")

#Function to turn "genres" columns from string into list
def string_to_list(genres):
    return eval(genres)

users["fav_genres"] = users["fav_genres"].apply(string_to_list)
anime["genre"] = anime["genre"].apply(string_to_list)
ratings["genre"] = ratings["genre"].apply(string_to_list)

In [77]:

users
user_vector = users.loc[:, ["experience","gender", "generation", "fav_genres", "fav_anime_period"]]
user_vector
# encoding users: 
# 1. encode fav_genres as bag of words: use term frequency vector for each
# 2. encode other features
# 3. join encoded genres with the rest of the vectorised features
# need to translate genres that user is asked to anime genres


,experience,gender,generation,fav_genres,fav_anime_period
0,Active,Female,Millenials,"[Comedy, Action, Shounen]",Classic
1,Active,Male,Millenials,"[Drama, Action, Sci-Fi]",Classic
2,Veteran,Male,Millenials,"[Comedy, Romance, School]",Classic
3,Veteran,Male,Millenials,"[Action, Power, Super]",Classic
4,Veteran,Female,Millenials,"[Action, Sci-Fi, Historical]",New-Gen
...,...,...,...,...,...
9297,Veteran,Male,Millenials,"[Ecchi, Comedy, Action]",New-Gen
9298,Veteran,Female,Gen Z,"[Action, Supernatural, Comedy]",New-Gen
9299,Veteran,Male,Millenials,"[Comedy, Adventure, Action]",Classic
9300,Regular,Female,Millenials,"[Comedy, Drama, Adventure]",Classic


In [75]:

anime_vector = anime.loc[:, ["airing","studio", "genre", "time_period", "fame"]]
anime_vector
#anime

,airing,studio,genre,time_period,fame
0,False,David Production,"[Comedy, Supernatural, Romance, Shounen]",New-Gen,famous
1,False,Gonzo,"[Comedy, Parody, Romance, School, Shounen]",Classic,semi-famous
2,False,Satelight,"[Comedy, Magic, School, Shoujo]",Classic,well-known
3,False,Hal Film Maker,"[Comedy, Drama, Magic, Romance, Fantasy]",Classic,semi-famous
4,False,J.C.Staff,"[Comedy, Drama, Romance, Shounen]",New-Gen,semi-famous
...,...,...,...,...,...
3502,False,Millepensee,"[Music, Drama]",New-Gen,niche
3503,False,Production I.G,"[Comedy, Sports, School, Shounen]",New-Gen,famous
3504,False,"Satelight, Production Reed","[Action, Mecha, Romance, Super Power, Supe...",Classic,well-known
3505,False,Toei Animation,"[Adventure, Fantasy]",Classic,unknown


In [73]:

ratings

,username,anime_id,title,genre,type,my_score,my_watched_episodes,aired_from_year,time_period
0,--FallenAngel--,164,Mononoke Hime,"[Action, Adventure, Fantasy]",Movie,10,1,1997,Classic
1,--FallenAngel--,5341,Ookami to Koushinryou II,"[Adventure, Historical, Romance, Fantasy]",TV,10,12,2009,Classic
2,--FallenAngel--,877,Nana,"[Music, Slice of Life, Comedy, Drama, Roma...",TV,10,47,2006,Classic
3,--FallenAngel--,3731,Itazura na Kiss,"[Comedy, Romance, Shoujo]",TV,10,25,2008,Classic
4,--FallenAngel--,1914,Saiunkoku Monogatari 2nd Season,"[Adventure, Comedy, Drama, Fantasy, Histor...",TV,10,39,2007,Classic
...,...,...,...,...,...,...,...,...,...
92927,zzs,4898,Kuroshitsuji,"[Action, Comedy, Demons, Fantasy, Historic...",TV,8,24,2008,Classic
92928,zzs,60,Chrno Crusade,"[Action, Historical, Demons, Supernatural, ...",TV,8,24,2003,Classic
92929,zzs,167,Scrapped Princess,"[Adventure, Comedy, Drama, Fantasy, Mecha,...",TV,8,24,2003,Classic
92930,zzs,1974,Glass no Kamen (2005),"[Drama, Shoujo]",TV,8,51,2005,Classic
